In [ ]:
# if needed, install the libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib oauth2client

In [1]:
import json
import yaml
import datetime
import base64
import email
from googleapiclient.discovery import build
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials
from google.auth.transport.requests import Request
from apiclient import discovery, errors

In [2]:
#you'll need these scopes
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [ ]:
# authenticate as your service account for your gcloud project
credentials = service_account.Credentials.from_service_account_file(
        '/path/to/file.json', scopes=SCOPES)
credentials.service_account_email

In [4]:
# get delegated access to a user's mail
user_email="someone@somewhere.com"
user_id="me"
try:    
    credentials = credentials.with_subject(user_email)
    gmail = build('gmail', 'v1', credentials=credentials)
except Exception as e:
    print(e)

In [ ]:
# historyId returned in the profile is useful for orienting to 
# where to start looking in history for actions on a message
gmail.users().getProfile(userId=user_id).execute()

In [ ]:
# get user's labels to orient
gmail.users().labels().list(userId=user_id).execute()

In [ ]:
# find messages matching query with a chosen subject
# query can be anything that works in gmail's search bar
query=r"""
from:someone@somewhere.com subject:'a subject you are looking for'
"""

messages=gmail.users().messages().list(userId=user_id,
                              maxResults=100,
                              includeSpamTrash=True,
                              q=query).execute()
for item in messages['messages']:
    message = gmail.users().messages().get(userId=user_id, id=item['id']).execute()
    print(f"id:{message['id']}, historyId: {message['historyId']}, labels: {message['labelIds']}")
    print(yaml.dump(message['payload']['headers']))
    


In [ ]:
# get all data elements of  particular message
message_id='somethingLike1a297881318f'
message=gmail.users().messages().get(userId=user_id, format='full',id=message_id).execute()
message

In [ ]:
#find history for a particular message

# set the message ID
message_id='somethingLike1a297881318f'

# for history ID, start at least a thousand back from what is returned from 'historyId' for a particular message
# and adjust accordingly
start_history_id=3461100 

history = (gmail.users().history().list(userId=user_id, startHistoryId=start_history_id).execute())
changes = history['history'] if 'history' in history else []
while 'nextPageToken' in history:
    page_token = history['nextPageToken']
    history = (gmail.users().history().list(userId=user_id,
                                    startHistoryId=start_history_id,
                                    pageToken=page_token).execute())
    current_historyId=history['historyId']
    changes.extend(history['history'])


print(f'current historyId:{current_historyId}')
for item in changes:
    if message_id in [i['id'] for i in item['messages']]:
        print(item)

# note you can only infer a relative timestamp from the history ID since there is no timestamp in the history record: 
# Please upvote/star https://issuetracker.google.com/issues/36759772
# but you can at least tell order of operations via the incrementing historyId

In [122]:
# get UTC date from a message 'internalDate' (epoch in milliseconds)
datetime.datetime.utcfromtimestamp(int(message['internalDate'])/1000).replace(tzinfo=datetime.timezone.utc).isoformat()

'2020-04-22T15:52:51+00:00'

In [ ]:
# get the raw message, decode it
message_id='somethingLike1a297881318f'
message=gmail.users().messages().get(userId=user_id, format='raw',id=message_id).execute()
mime_message=base64.urlsafe_b64decode(message['raw'].encode('ASCII'))
email_message=email.message_from_bytes(mime_message)
print(email_message.as_string())

In [44]:
base64.b64decode('dGVzdAo=')

b'test\n'